# Minnesota Standard Deduction Reform Analysis (2025)

This notebook analyzes the impact of increasing Minnesota's standard deduction.

## Baseline (Current Law 2025)
- Single/Separate: $14,950
- Head of Household: $22,500
- Joint/Surviving Spouse: $29,900

## Reform
- Single/Separate: $20,000
- Head of Household: $30,000
- Joint/Surviving Spouse: $40,000

## Focus: Loser Population Analysis
While most taxpayers benefit from higher standard deductions, a small group may be worse off due to:
1. **Federal AMT interactions** - Higher state deductions can increase federal AMT liability
2. **Itemized deduction interactions** - Those who itemize may see reduced benefit

In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import numpy as np

MN_DATASET = "hf://policyengine/policyengine-us-data/states/MN.h5"

## Define Reform

In [2]:
def create_mn_standard_deduction_reform():
    reform = Reform.from_dict(
        {
            "gov.states.mn.tax.income.deductions.standard.base.SINGLE": {
                "2025-01-01.2100-12-31": 20000
            },
            "gov.states.mn.tax.income.deductions.standard.base.SEPARATE": {
                "2025-01-01.2100-12-31": 20000
            },
            "gov.states.mn.tax.income.deductions.standard.base.HEAD_OF_HOUSEHOLD": {
                "2025-01-01.2100-12-31": 30000
            },
            "gov.states.mn.tax.income.deductions.standard.base.JOINT": {
                "2025-01-01.2100-12-31": 40000
            },
            "gov.states.mn.tax.income.deductions.standard.base.SURVIVING_SPOUSE": {
                "2025-01-01.2100-12-31": 40000
            },
        },
        country_id="us",
    )
    return reform

print("Reform defined!")
print("\nReform details:")
print("  - Single/Separate: $14,950 -> $20,000 (+$5,050)")
print("  - Head of Household: $22,500 -> $30,000 (+$7,500)")
print("  - Joint/Surviving Spouse: $29,900 -> $40,000 (+$10,100)")

Reform defined!

Reform details:
  - Single/Separate: $14,950 -> $20,000 (+$5,050)
  - Head of Household: $22,500 -> $30,000 (+$7,500)
  - Joint/Surviving Spouse: $29,900 -> $40,000 (+$10,100)


## Load Simulations

In [3]:
print("Loading baseline (current law)...")
baseline = Microsimulation(dataset=MN_DATASET)
print("Baseline loaded")

print("\nLoading reform...")
reform = create_mn_standard_deduction_reform()
reform_sim = Microsimulation(dataset=MN_DATASET, reform=reform)
print("Reform loaded")

print("\n" + "="*60)
print("All simulations ready!")
print("="*60)

Loading baseline (current law)...
Baseline loaded

Loading reform...
Reform loaded

All simulations ready!


## Overall Reform Impact

In [4]:
# Calculate overall impact
baseline_hh_income = np.array(baseline.calculate("household_net_income", period=2025, map_to="household"))
reform_hh_income = np.array(reform_sim.calculate("household_net_income", period=2025, map_to="household"))
household_weight = np.array(baseline.calculate("household_weight", period=2025))

income_change = reform_hh_income - baseline_hh_income
total_cost = (income_change * household_weight).sum()

# Winners and losers
winners_mask = income_change > 1
losers_mask = income_change < -1

winners_count = household_weight[winners_mask].sum()
losers_count = household_weight[losers_mask].sum()
total_households = household_weight.sum()

print("="*70)
print("MN STANDARD DEDUCTION REFORM - OVERALL IMPACT")
print("="*70)
print(f"Net cost to state:          ${total_cost/1e6:,.2f}M")
print(f"\nHouseholds benefitting:     {winners_count:,.0f} ({winners_count/total_households*100:.2f}%)")
print(f"Households losing:          {losers_count:,.0f} ({losers_count/total_households*100:.2f}%)")
print(f"Total households:           {total_households:,.0f}")

if winners_mask.sum() > 0:
    avg_gain = np.average(income_change[winners_mask], weights=household_weight[winners_mask])
    print(f"\nAvg gain (winners):         ${avg_gain:,.2f}")
if losers_mask.sum() > 0:
    avg_loss = np.average(income_change[losers_mask], weights=household_weight[losers_mask])
    print(f"Avg loss (losers):          ${avg_loss:,.2f}")
print("="*70)

MN STANDARD DEDUCTION REFORM - OVERALL IMPACT
Net cost to state:          $506.68M

Households benefitting:     770,620 (61.41%)
Households losing:          45,249 (3.61%)
Total households:           1,254,857

Avg gain (winners):         $663.90
Avg loss (losers):          $-109.12


## Loser Population Deep Dive

In [5]:
# Get tax unit level data for detailed analysis
tu_id = np.array(baseline.calculate("tax_unit_id", period=2025, map_to="tax_unit"))
tu_weight = np.array(baseline.calculate("tax_unit_weight", period=2025))

# Income metrics
baseline_tu_income = np.array(baseline.calculate("household_net_income", period=2025, map_to="tax_unit"))
reform_tu_income = np.array(reform_sim.calculate("household_net_income", period=2025, map_to="tax_unit"))
tu_income_change = reform_tu_income - baseline_tu_income

# Key variables to understand losers
agi = np.array(baseline.calculate("adjusted_gross_income", period=2025, map_to="tax_unit"))
filing_status = np.array(baseline.calculate("filing_status", period=2025, map_to="tax_unit"))

# AMT
baseline_amt = np.array(baseline.calculate("alternative_minimum_tax", period=2025, map_to="tax_unit"))
reform_amt = np.array(reform_sim.calculate("alternative_minimum_tax", period=2025, map_to="tax_unit"))
amt_change = reform_amt - baseline_amt

# Itemized vs Standard deduction choice
tax_unit_itemizes = np.array(baseline.calculate("tax_unit_itemizes", period=2025, map_to="tax_unit"))

# MN-specific
baseline_mn_income_tax = np.array(baseline.calculate("mn_income_tax", period=2025, map_to="tax_unit"))
reform_mn_income_tax = np.array(reform_sim.calculate("mn_income_tax", period=2025, map_to="tax_unit"))
mn_tax_change = reform_mn_income_tax - baseline_mn_income_tax

# Federal income tax
baseline_fed_tax = np.array(baseline.calculate("income_tax", period=2025, map_to="tax_unit"))
reform_fed_tax = np.array(reform_sim.calculate("income_tax", period=2025, map_to="tax_unit"))
fed_tax_change = reform_fed_tax - baseline_fed_tax

print("Tax unit level data loaded for analysis")

Tax unit level data loaded for analysis


In [6]:
# Identify losers at tax unit level
tu_losers_mask = tu_income_change < -1

print("="*70)
print("LOSER POPULATION CHARACTERISTICS")
print("="*70)

losers_count_tu = tu_weight[tu_losers_mask].sum()
total_tu = tu_weight.sum()
print(f"Tax units losing income:    {losers_count_tu:,.0f} ({losers_count_tu/total_tu*100:.2f}%)")

if tu_losers_mask.sum() > 0:
    # Average loss
    avg_loss_tu = np.average(tu_income_change[tu_losers_mask], weights=tu_weight[tu_losers_mask])
    print(f"Average income loss:        ${avg_loss_tu:,.2f}")
    
    # AGI distribution of losers
    avg_agi_losers = np.average(agi[tu_losers_mask], weights=tu_weight[tu_losers_mask])
    print(f"\nAverage AGI of losers:      ${avg_agi_losers:,.0f}")
    
    # Itemization status
    itemizers_among_losers = tu_weight[tu_losers_mask & tax_unit_itemizes].sum()
    pct_itemizers = itemizers_among_losers / losers_count_tu * 100 if losers_count_tu > 0 else 0
    print(f"\nLosers who itemize:         {itemizers_among_losers:,.0f} ({pct_itemizers:.1f}%)")
    
    # AMT payers among losers
    amt_payers_baseline = tu_weight[tu_losers_mask & (baseline_amt > 0)].sum()
    pct_amt = amt_payers_baseline / losers_count_tu * 100 if losers_count_tu > 0 else 0
    print(f"Losers paying AMT (baseline): {amt_payers_baseline:,.0f} ({pct_amt:.1f}%)")
    
    # AMT increase for losers
    amt_increased = tu_weight[tu_losers_mask & (amt_change > 0)].sum()
    pct_amt_increase = amt_increased / losers_count_tu * 100 if losers_count_tu > 0 else 0
    print(f"Losers with AMT increase:   {amt_increased:,.0f} ({pct_amt_increase:.1f}%)")

print("="*70)

LOSER POPULATION CHARACTERISTICS
Tax units losing income:    53,091 (2.50%)
Average income loss:        $-93.01

Average AGI of losers:      $278,523

Losers who itemize:         52,920 (99.7%)
Losers paying AMT (baseline): 19 (0.0%)
Losers with AMT increase:   0 (0.0%)


## Why Are They Worse Off? - Tax Component Analysis

In [7]:
if tu_losers_mask.sum() > 0:
    print("="*70)
    print("TAX CHANGE BREAKDOWN FOR LOSERS")
    print("="*70)
    
    # Average tax changes for losers
    avg_mn_tax_change = np.average(mn_tax_change[tu_losers_mask], weights=tu_weight[tu_losers_mask])
    avg_fed_tax_change = np.average(fed_tax_change[tu_losers_mask], weights=tu_weight[tu_losers_mask])
    avg_amt_change = np.average(amt_change[tu_losers_mask], weights=tu_weight[tu_losers_mask])
    
    print(f"Average MN income tax change:    ${avg_mn_tax_change:,.2f}")
    print(f"Average Federal tax change:      ${avg_fed_tax_change:,.2f}")
    print(f"Average AMT change:              ${avg_amt_change:,.2f}")
    
    print("\n" + "-"*70)
    print("Interpretation:")
    if avg_mn_tax_change < 0:
        print(f"  - MN tax DECREASED by ${abs(avg_mn_tax_change):,.2f} (expected from higher std deduction)")
    else:
        print(f"  - MN tax INCREASED by ${avg_mn_tax_change:,.2f} (unexpected!)")
    
    if avg_fed_tax_change > 0:
        print(f"  - Federal tax INCREASED by ${avg_fed_tax_change:,.2f}")
        print(f"    -> This could be due to reduced SALT deduction benefit")
    
    if avg_amt_change > 0:
        print(f"  - AMT INCREASED by ${avg_amt_change:,.2f}")
        print(f"    -> Higher state deduction reduces state tax, increasing AMT exposure")
    
    print("="*70)

TAX CHANGE BREAKDOWN FOR LOSERS
Average MN income tax change:    $-13.53
Average Federal tax change:      $106.54
Average AMT change:              $-0.00

----------------------------------------------------------------------
Interpretation:
  - MN tax DECREASED by $13.53 (expected from higher std deduction)
  - Federal tax INCREASED by $106.54
    -> This could be due to reduced SALT deduction benefit


## Loser Categories Analysis

In [8]:
if tu_losers_mask.sum() > 0:
    print("="*70)
    print("CATEGORIZING LOSERS BY CAUSE")
    print("="*70)
    
    # Category 1: AMT-driven losses (AMT increased)
    amt_driven = tu_losers_mask & (amt_change > 1)
    amt_driven_count = tu_weight[amt_driven].sum()
    
    # Category 2: Itemizers (may lose SALT benefit)
    itemizer_driven = tu_losers_mask & tax_unit_itemizes & (amt_change <= 1)
    itemizer_driven_count = tu_weight[itemizer_driven].sum()
    
    # Category 3: Other/Unknown
    other = tu_losers_mask & ~amt_driven & ~itemizer_driven
    other_count = tu_weight[other].sum()
    
    print(f"\n1. AMT-Driven Losses:           {amt_driven_count:,.0f} ({amt_driven_count/losers_count_tu*100:.1f}%)")
    if amt_driven.sum() > 0:
        avg_amt_inc = np.average(amt_change[amt_driven], weights=tu_weight[amt_driven])
        avg_loss_amt = np.average(tu_income_change[amt_driven], weights=tu_weight[amt_driven])
        print(f"   Avg AMT increase: ${avg_amt_inc:,.2f}")
        print(f"   Avg income loss:  ${avg_loss_amt:,.2f}")
    
    print(f"\n2. Itemizer-Related Losses:     {itemizer_driven_count:,.0f} ({itemizer_driven_count/losers_count_tu*100:.1f}%)")
    if itemizer_driven.sum() > 0:
        avg_loss_item = np.average(tu_income_change[itemizer_driven], weights=tu_weight[itemizer_driven])
        print(f"   Avg income loss:  ${avg_loss_item:,.2f}")
    
    print(f"\n3. Other/Unknown:               {other_count:,.0f} ({other_count/losers_count_tu*100:.1f}%)")
    if other.sum() > 0:
        avg_loss_other = np.average(tu_income_change[other], weights=tu_weight[other])
        print(f"   Avg income loss:  ${avg_loss_other:,.2f}")
    
    print("="*70)

CATEGORIZING LOSERS BY CAUSE

1. AMT-Driven Losses:           0 (0.0%)

2. Itemizer-Related Losses:     52,920 (99.7%)
   Avg income loss:  $-93.13

3. Other/Unknown:               171 (0.3%)
   Avg income loss:  $-54.97


## AGI Distribution of Losers

In [9]:
if tu_losers_mask.sum() > 0:
    print("="*70)
    print("LOSERS BY AGI BRACKET")
    print("="*70)
    
    brackets = [
        (0, 50000, "$0-$50k"),
        (50000, 100000, "$50k-$100k"),
        (100000, 200000, "$100k-$200k"),
        (200000, 500000, "$200k-$500k"),
        (500000, 1000000, "$500k-$1M"),
        (1000000, float('inf'), "$1M+"),
    ]
    
    print(f"{'AGI Bracket':<15} {'Losers':>12} {'% of Losers':>14} {'Avg Loss':>12} {'Avg AMT Chg':>12}")
    print("-"*70)
    
    for lower, upper, label in brackets:
        bracket_losers = tu_losers_mask & (agi >= lower) & (agi < upper)
        bracket_count = tu_weight[bracket_losers].sum()
        
        if bracket_count > 0:
            pct = bracket_count / losers_count_tu * 100
            avg_loss = np.average(tu_income_change[bracket_losers], weights=tu_weight[bracket_losers])
            avg_amt = np.average(amt_change[bracket_losers], weights=tu_weight[bracket_losers])
            print(f"{label:<15} {bracket_count:>12,.0f} {pct:>13.1f}% ${avg_loss:>10,.0f} ${avg_amt:>10,.0f}")
        else:
            print(f"{label:<15} {0:>12} {0:>13.1f}% {'N/A':>12} {'N/A':>12}")
    
    print("="*70)

LOSERS BY AGI BRACKET
AGI Bracket           Losers    % of Losers     Avg Loss  Avg AMT Chg
----------------------------------------------------------------------
$0-$50k                  204           0.4% $       -51 $         0
$50k-$100k            11,788          22.2% $       -61 $         0
$100k-$200k            8,432          15.9% $       -75 $         0
$200k-$500k           24,723          46.6% $      -134 $         0
$500k-$1M              7,872          14.8% $       -32 $        -0
$1M+                      72           0.1% $       -72 $         0


## Filing Status Distribution of Losers

In [10]:
if tu_losers_mask.sum() > 0:
    print("="*70)
    print("LOSERS BY FILING STATUS")
    print("="*70)
    
    # Filing status is returned as string enum values
    status_names = ["SINGLE", "JOINT", "SEPARATE", "HEAD_OF_HOUSEHOLD", "SURVIVING_SPOUSE"]
    status_display = {
        "SINGLE": "Single",
        "JOINT": "Joint",
        "SEPARATE": "Separate",
        "HEAD_OF_HOUSEHOLD": "Head of Household",
        "SURVIVING_SPOUSE": "Surviving Spouse"
    }
    
    print(f"{'Filing Status':<20} {'Losers':>12} {'% of Losers':>14} {'Avg Loss':>12}")
    print("-"*70)
    
    for status_code in status_names:
        status_losers = tu_losers_mask & (filing_status == status_code)
        status_count = tu_weight[status_losers].sum()
        
        if status_count > 0:
            pct = status_count / losers_count_tu * 100
            avg_loss = np.average(tu_income_change[status_losers], weights=tu_weight[status_losers])
            print(f"{status_display[status_code]:<20} {status_count:>12,.0f} {pct:>13.1f}% ${avg_loss:>10,.0f}")
    
    print("="*70)

LOSERS BY FILING STATUS
Filing Status              Losers    % of Losers     Avg Loss
----------------------------------------------------------------------
Single                     20,433          38.5% $       -41
Joint                      32,590          61.4% $      -126
Separate                        1           0.0% $       -41
Head of Household              67           0.1% $       -91
Surviving Spouse                0           0.0% $      -119


## Detailed AMT Analysis for Losers

In [11]:
# Get additional AMT-related variables
baseline_amt_income = np.array(baseline.calculate("amt_income", period=2025, map_to="tax_unit"))
reform_amt_income = np.array(reform_sim.calculate("amt_income", period=2025, map_to="tax_unit"))

if tu_losers_mask.sum() > 0:
    print("="*70)
    print("AMT MECHANISM ANALYSIS FOR LOSERS")
    print("="*70)
    
    # Focus on losers with AMT increase
    amt_losers = tu_losers_mask & (amt_change > 0)
    amt_losers_count = tu_weight[amt_losers].sum()
    
    if amt_losers.sum() > 0:
        print(f"\nTax units with AMT increase: {amt_losers_count:,.0f}")
        
        # AMT income change
        amt_income_change = reform_amt_income - baseline_amt_income
        avg_amti_change = np.average(amt_income_change[amt_losers], weights=tu_weight[amt_losers])
        
        print(f"\nAvg AMT Income change:       ${avg_amti_change:,.2f}")
        
        print("\n" + "-"*70)
        print("Explanation:")
        print("  When MN standard deduction increases:")
        print("  1. MN taxable income decreases")
        print("  2. MN income tax liability decreases")
        print("  3. SALT deduction (if itemizing) may decrease")
        print("  4. Federal taxable income may increase slightly")
        print("  5. For AMT purposes, state taxes are not deductible")
        print("  6. Lower state taxes can push filers into AMT territory")
    else:
        print("\nNo losers with AMT increase found.")
    
    print("="*70)

AMT MECHANISM ANALYSIS FOR LOSERS

No losers with AMT increase found.


## SALT Deduction Analysis for Losers

In [12]:
# Get SALT deduction data
try:
    baseline_salt = np.array(baseline.calculate("salt_deduction", period=2025, map_to="tax_unit"))
    reform_salt = np.array(reform_sim.calculate("salt_deduction", period=2025, map_to="tax_unit"))
    salt_change = reform_salt - baseline_salt
    
    if tu_losers_mask.sum() > 0:
        print("="*70)
        print("SALT DEDUCTION ANALYSIS FOR LOSERS")
        print("="*70)
        
        # Losers who itemize
        itemizing_losers = tu_losers_mask & tax_unit_itemizes
        itemizing_losers_count = tu_weight[itemizing_losers].sum()
        
        if itemizing_losers.sum() > 0:
            print(f"\nItemizing losers: {itemizing_losers_count:,.0f}")
            
            avg_salt_change = np.average(salt_change[itemizing_losers], weights=tu_weight[itemizing_losers])
            avg_baseline_salt = np.average(baseline_salt[itemizing_losers], weights=tu_weight[itemizing_losers])
            
            print(f"Avg baseline SALT:           ${avg_baseline_salt:,.2f}")
            print(f"Avg SALT change:             ${avg_salt_change:,.2f}")
            
            print("\n" + "-"*70)
            print("Note: SALT deduction is capped at $10,000 for federal purposes.")
            print("Changes in state tax liability affect SALT if below the cap.")
        
        print("="*70)
except:
    print("SALT deduction variable not available - skipping analysis")

SALT DEDUCTION ANALYSIS FOR LOSERS

Itemizing losers: 52,920
Avg baseline SALT:           $17,567.71
Avg SALT change:             $-549.29

----------------------------------------------------------------------
Note: SALT deduction is capped at $10,000 for federal purposes.
Changes in state tax liability affect SALT if below the cap.


## Sample Loser Profiles

In [13]:
if tu_losers_mask.sum() > 0:
    print("="*90)
    print("SAMPLE LOSER PROFILES (Top 10 by Weighted Loss)")
    print("="*90)
    
    # Create dataframe for losers
    loser_indices = np.where(tu_losers_mask)[0]
    
    df_losers = pd.DataFrame({
        'index': loser_indices,
        'weight': tu_weight[loser_indices],
        'agi': agi[loser_indices],
        'filing_status': filing_status[loser_indices],
        'itemizes': tax_unit_itemizes[loser_indices],
        'income_change': tu_income_change[loser_indices],
        'mn_tax_change': mn_tax_change[loser_indices],
        'fed_tax_change': fed_tax_change[loser_indices],
        'amt_change': amt_change[loser_indices],
        'baseline_amt': baseline_amt[loser_indices],
        'reform_amt': reform_amt[loser_indices]
    })
    
    # Sort by weighted loss (loss * weight)
    df_losers['weighted_loss'] = df_losers['income_change'] * df_losers['weight']
    df_losers = df_losers.sort_values('weighted_loss').head(10)
    
    # Filing status display mapping (string enum values)
    status_map = {
        'SINGLE': 'Single',
        'JOINT': 'Joint', 
        'SEPARATE': 'Separate',
        'HEAD_OF_HOUSEHOLD': 'HoH',
        'SURVIVING_SPOUSE': 'Surv Sp'
    }
    
    print(f"\n{'#':<3} {'AGI':>12} {'Status':<10} {'Itemizes':<9} {'Loss':>10} {'MN Tax':>10} {'Fed Tax':>10} {'AMT Chg':>10}")
    print("-"*90)
    
    for i, (_, row) in enumerate(df_losers.iterrows(), 1):
        status = status_map.get(str(row['filing_status']), 'Unknown')
        itemizes = 'Yes' if row['itemizes'] else 'No'
        print(f"{i:<3} ${row['agi']:>10,.0f} {status:<10} {itemizes:<9} ${row['income_change']:>9,.0f} ${row['mn_tax_change']:>9,.0f} ${row['fed_tax_change']:>9,.0f} ${row['amt_change']:>9,.0f}")
    
    print("="*90)

SAMPLE LOSER PROFILES (Top 10 by Weighted Loss)

#            AGI Status     Itemizes        Loss     MN Tax    Fed Tax    AMT Chg
------------------------------------------------------------------------------------------
1   $   379,259 Joint      Yes       $     -206 $        0 $      206 $        0
2   $   379,259 Joint      Yes       $     -206 $        0 $      206 $        0
3   $   379,259 Joint      Yes       $     -206 $        0 $      206 $        0
4   $   294,776 Joint      Yes       $     -190 $        0 $      190 $        0
5   $   379,259 Joint      Yes       $     -206 $        0 $      206 $        0
6   $   379,259 Joint      Yes       $     -206 $        0 $      206 $        0
7   $   379,259 Joint      Yes       $     -206 $        0 $      206 $        0
8   $   294,776 Joint      Yes       $     -190 $        0 $      190 $        0
9   $   379,259 Joint      Yes       $     -206 $        0 $      206 $        0
10  $   294,776 Joint      Yes       $     -190 $

## Summary and Conclusions

In [14]:
print("="*70)
print("SUMMARY: WHY SOME TAXPAYERS LOSE FROM HIGHER MN STANDARD DEDUCTION")
print("="*70)

print("""
The Minnesota standard deduction reform increases deductions for all filers:
  - Single/Separate: $14,950 -> $20,000 (+$5,050)
  - Head of Household: $22,500 -> $30,000 (+$7,500)
  - Joint/Surviving: $29,900 -> $40,000 (+$10,100)

MOST TAXPAYERS BENEFIT because:
  - Higher state deduction = Lower MN taxable income = Lower MN tax

SOME TAXPAYERS LOSE due to:

1. FEDERAL AMT INTERACTION
   - State/local taxes are NOT deductible for AMT purposes
   - Lower state tax liability doesn't reduce AMT income
   - Some filers are pushed further into AMT territory
   - The AMT increase can exceed the MN tax savings

2. ITEMIZED DEDUCTION INTERACTION
   - Filers who itemize may see reduced benefit
   - Lower MN tax = Lower SALT deduction (if not at $10k cap)
   - This can increase federal taxable income
   - Combined federal+state effect can be negative

3. PROGRAM INTERACTIONS
   - Changes in net income can affect benefit eligibility
   - Some benefit phase-outs may be triggered
""")

if tu_losers_mask.sum() > 0:
    total_losers = tu_weight[tu_losers_mask].sum()
    total_loss = (tu_income_change[tu_losers_mask] * tu_weight[tu_losers_mask]).sum()
    
    print(f"QUANTIFIED IMPACT:")
    print(f"  - Total losers: {total_losers:,.0f} tax units")
    print(f"  - Total losses: ${total_loss/1e6:,.2f}M")
    print(f"  - As % of reform cost: {abs(total_loss/total_cost)*100:.1f}%")

print("="*70)

SUMMARY: WHY SOME TAXPAYERS LOSE FROM HIGHER MN STANDARD DEDUCTION

The Minnesota standard deduction reform increases deductions for all filers:
  - Single/Separate: $14,950 -> $20,000 (+$5,050)
  - Head of Household: $22,500 -> $30,000 (+$7,500)
  - Joint/Surviving: $29,900 -> $40,000 (+$10,100)

MOST TAXPAYERS BENEFIT because:
  - Higher state deduction = Lower MN taxable income = Lower MN tax

SOME TAXPAYERS LOSE due to:

1. FEDERAL AMT INTERACTION
   - State/local taxes are NOT deductible for AMT purposes
   - Lower state tax liability doesn't reduce AMT income
   - Some filers are pushed further into AMT territory
   - The AMT increase can exceed the MN tax savings

2. ITEMIZED DEDUCTION INTERACTION
   - Filers who itemize may see reduced benefit
   - Lower MN tax = Lower SALT deduction (if not at $10k cap)
   - This can increase federal taxable income
   - Combined federal+state effect can be negative

3. PROGRAM INTERACTIONS
   - Changes in net income can affect benefit eligibil

## Diagnostic: SALT vs MN Tax Change Consistency Check

In [15]:
# Diagnostic: Check if SALT change tracks MN tax change correctly
# If there's no bug, SALT change should approximately equal MN tax change for itemizers

print("="*70)
print("DIAGNOSTIC: SALT vs MN TAX CHANGE CONSISTENCY")
print("="*70)

# Get the changes for losers
mn_tax_diff = mn_tax_change[tu_losers_mask]
salt_diff = salt_change[tu_losers_mask]
loser_weights = tu_weight[tu_losers_mask]

avg_mn_tax_change = np.average(mn_tax_diff, weights=loser_weights)
avg_salt_change = np.average(salt_diff, weights=loser_weights)

print(f"\nFor LOSERS:")
print(f"  Avg MN tax change:     ${avg_mn_tax_change:,.2f}")
print(f"  Avg SALT change:       ${avg_salt_change:,.2f}")
print(f"  Difference:            ${avg_salt_change - avg_mn_tax_change:,.2f}")

# Check correlation
if len(mn_tax_diff) > 1:
    correlation = np.corrcoef(mn_tax_diff, salt_diff)[0,1]
    print(f"  Correlation:           {correlation:.3f}")

print("\n" + "-"*70)
print("EXPECTED: If no bug, SALT change ≈ MN tax change for itemizers")
print("ACTUAL: SALT is changing MORE than MN tax - suggests a bug")
print("-"*70)

# Look at specific cases where MN tax = 0 but SALT changed
zero_mn_change = tu_losers_mask & (np.abs(mn_tax_change) < 1)
if zero_mn_change.sum() > 0:
    print(f"\nTax units with MN tax change ≈ $0 but losing: {tu_weight[zero_mn_change].sum():,.0f}")
    avg_salt_zero_mn = np.average(salt_change[zero_mn_change], weights=tu_weight[zero_mn_change])
    print(f"  Their avg SALT change: ${avg_salt_zero_mn:,.2f}")
    print(f"  -> BUG INDICATOR: SALT changed when MN tax didn't!")

# Compare across ALL tax units (not just losers)
print("\n" + "="*70)
print("FULL POPULATION CHECK")
print("="*70)

itemizers_mask = tax_unit_itemizes
all_mn_change = np.average(mn_tax_change[itemizers_mask], weights=tu_weight[itemizers_mask])
all_salt_change = np.average(salt_change[itemizers_mask], weights=tu_weight[itemizers_mask])

print(f"\nFor ALL ITEMIZERS:")
print(f"  Avg MN tax change:     ${all_mn_change:,.2f}")
print(f"  Avg SALT change:       ${all_salt_change:,.2f}")
print(f"  Difference:            ${all_salt_change - all_mn_change:,.2f}")

if np.abs(all_salt_change - all_mn_change) > 10:
    print(f"\n⚠️  WARNING: SALT and MN tax changes don't match!")
    print(f"   This suggests SALT may not be correctly tracking actual MN tax paid.")
else:
    print(f"\n✓ SALT and MN tax changes are consistent")

DIAGNOSTIC: SALT vs MN TAX CHANGE CONSISTENCY

For LOSERS:
  Avg MN tax change:     $-13.53
  Avg SALT change:       $-547.52
  Difference:            $-533.99
  Correlation:           0.238

----------------------------------------------------------------------
EXPECTED: If no bug, SALT change ≈ MN tax change for itemizers
ACTUAL: SALT is changing MORE than MN tax - suggests a bug
----------------------------------------------------------------------

Tax units with MN tax change ≈ $0 but losing: 44,836
  Their avg SALT change: $-643.16
  -> BUG INDICATOR: SALT changed when MN tax didn't!

FULL POPULATION CHECK

For ALL ITEMIZERS:
  Avg MN tax change:     $-279.66
  Avg SALT change:       $-284.90
  Difference:            $-5.24

✓ SALT and MN tax changes are consistent


In [16]:
# Deeper diagnostic: Check MN deduction values and itemization choice
print("="*70)
print("DIAGNOSTIC: MN DEDUCTION CHOICE")
print("="*70)

# Get MN deduction-related variables
try:
    baseline_mn_deductions = np.array(baseline.calculate("mn_deductions", period=2025, map_to="tax_unit"))
    reform_mn_deductions = np.array(reform_sim.calculate("mn_deductions", period=2025, map_to="tax_unit"))
    mn_ded_change = reform_mn_deductions - baseline_mn_deductions
    
    print(f"\nMN Deductions (total used):")
    print(f"  Baseline avg (losers): ${np.average(baseline_mn_deductions[tu_losers_mask], weights=loser_weights):,.2f}")
    print(f"  Reform avg (losers):   ${np.average(reform_mn_deductions[tu_losers_mask], weights=loser_weights):,.2f}")
    print(f"  Change (losers):       ${np.average(mn_ded_change[tu_losers_mask], weights=loser_weights):,.2f}")
except Exception as e:
    print(f"Could not get mn_deductions: {e}")

# Check if MN has itemization tracking
try:
    baseline_mn_itemizes = np.array(baseline.calculate("mn_itemizes", period=2025, map_to="tax_unit"))
    reform_mn_itemizes = np.array(reform_sim.calculate("mn_itemizes", period=2025, map_to="tax_unit"))
    
    # How many losers itemize on MN?
    losers_mn_itemize_baseline = tu_weight[tu_losers_mask & baseline_mn_itemizes].sum()
    losers_mn_itemize_reform = tu_weight[tu_losers_mask & reform_mn_itemizes].sum()
    
    print(f"\nMN Itemization Status (losers):")
    print(f"  Itemizing on MN (baseline): {losers_mn_itemize_baseline:,.0f} ({losers_mn_itemize_baseline/losers_count_tu*100:.1f}%)")
    print(f"  Itemizing on MN (reform):   {losers_mn_itemize_reform:,.0f} ({losers_mn_itemize_reform/losers_count_tu*100:.1f}%)")
    
    # Did any losers SWITCH from MN itemizing to MN standard?
    switched_to_standard = tu_losers_mask & baseline_mn_itemizes & ~reform_mn_itemizes
    switched_count = tu_weight[switched_to_standard].sum()
    print(f"  Switched to MN standard:    {switched_count:,.0f}")
except Exception as e:
    print(f"Could not get mn_itemizes: {e}")

# Check the actual SALT components
print("\n" + "-"*70)
print("SALT COMPONENTS CHECK")
print("-"*70)

try:
    # State income tax component of SALT
    baseline_state_income_tax = np.array(baseline.calculate("state_income_tax", period=2025, map_to="tax_unit"))
    reform_state_income_tax = np.array(reform_sim.calculate("state_income_tax", period=2025, map_to="tax_unit"))
    state_tax_change = reform_state_income_tax - baseline_state_income_tax
    
    print(f"\nState Income Tax (all states, for SALT):")
    print(f"  Avg change (losers): ${np.average(state_tax_change[tu_losers_mask], weights=loser_weights):,.2f}")
    print(f"  This should equal MN tax change: ${avg_mn_tax_change:,.2f}")
    
    if np.abs(np.average(state_tax_change[tu_losers_mask], weights=loser_weights) - avg_mn_tax_change) > 1:
        print(f"  ⚠️  MISMATCH between state_income_tax and mn_income_tax!")
except Exception as e:
    print(f"Could not get state_income_tax: {e}")

print("="*70)

DIAGNOSTIC: MN DEDUCTION CHOICE

MN Deductions (total used):
  Baseline avg (losers): $54,443.61
  Reform avg (losers):   $54,608.21
  Change (losers):       $164.59
Could not get mn_itemizes: Variable mn_itemizes does not exist.

----------------------------------------------------------------------
SALT COMPONENTS CHECK
----------------------------------------------------------------------

State Income Tax (all states, for SALT):
  Avg change (losers): $-13.53
  This should equal MN tax change: $-13.53
